In [ ]:
import cv2
import numpy as np
from PIL import ImageFont, ImageDraw, Image
import datetime

cap = cv2.VideoCapture(0)
font_path = "C:\\Windows\\Fonts\\Arial.ttf"
font_size = 24
font = ImageFont.truetype(font_path, font_size)


ret, first_frame = cap.read()
first_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
first_gray = cv2.GaussianBlur(first_gray, (21, 21), 0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

 
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (21, 21), 0)

 
    frameDelta = cv2.absdiff(first_gray, gray)
    thresh = cv2.threshold(frameDelta, 25, 255, cv2.THRESH_BINARY)[1]

  
    thresh = cv2.dilate(thresh, None, iterations=2)
    cnts, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


    total_area = sum(cv2.contourArea(c) for c in cnts)
    h, w = frame.shape[:2]
    coverage_ratio = total_area / (h * w)


    mean_brightness = np.mean(gray)

  
    frame_pil = Image.fromarray(frame)
    draw = ImageDraw.Draw(frame_pil)
    draw.text((10, 10), "Kamera Kapatılma Tespiti", font=font, fill=(0, 0, 255))

  
    if coverage_ratio > 0.08 and mean_brightness < 75:
        now = datetime.datetime.now()
        time_string = now.strftime("%Y-%m-%d %H:%M:%S")
        draw.text((w // 2 - 200, h // 2), "Kamera Kapatıldı!", font=font, fill=(0, 0, 255))
        draw.text((w // 2 - 200, h // 2 + font_size), time_string, font=font, fill=(0, 0, 255))

    frame = np.array(frame_pil)

   
    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
